## Basic Structured Operations


### Step 1: Initialize PySpark Session


In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, lit, desc, avg, concat
from pyspark.sql.types import StructType, StructField, StringType, IntegerType



# Create a Spark session
spark = SparkSession.builder.appName("day2").getOrCreate()


### Step 2: Load the Dataset


In [4]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "occupation.csv"  # Replace with the actual path
occupation = spark.read.csv(data_path, header=True, inferSchema=True)


In [5]:
occupation.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



### Problem 1: Selecting Specific Columns
Problem: Select the "user_id," "age," and "occupation" columns from the occupation DataFrame.

In [6]:
occupation.select("user_id","age","occupation").show()

+-------+---+-------------+
|user_id|age|   occupation|
+-------+---+-------------+
|      1| 24|   technician|
|      2| 53|        other|
|      3| 23|       writer|
|      4| 24|   technician|
|      5| 33|        other|
|      6| 42|    executive|
|      7| 57|administrator|
|      8| 36|administrator|
|      9| 29|      student|
|     10| 53|       lawyer|
|     11| 39|        other|
|     12| 28|        other|
|     13| 47|     educator|
|     14| 45|    scientist|
|     15| 49|     educator|
|     16| 21|entertainment|
|     17| 30|   programmer|
|     18| 35|        other|
|     19| 40|    librarian|
|     20| 42|    homemaker|
+-------+---+-------------+
only showing top 20 rows



### Problem 2: Filtering Rows based on Condition
Problem: Find the users who are older than 30 years from the occupation DataFrame.

In [15]:
occupation.select("*").filter(col("age")>30).show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      2| 53|     F|        other|   94043|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemaker|   95660|
|     25| 39|     M|     engineer|   55107|
|     26| 49|     M|     engineer|   21044|
|     27| 40|     F|    librarian|   30030|
|     28| 32|     M|       writer|   55369|
|     29| 41|     M|   programmer|   94043|
|     34| 38|     F|administrator|   42141|
|     39| 41|     M|entertainmen

### Problem 3: Counting and Grouping
Problem: Count the number of users in each occupation from the occupation DataFrame.

In [16]:
occupation.groupBy("occupation").count().show()

+-------------+-----+
|   occupation|count|
+-------------+-----+
|    librarian|   51|
|      retired|   14|
|       lawyer|   12|
|         none|    9|
|       writer|   45|
|   programmer|   66|
|    marketing|   26|
|        other|  105|
|    executive|   32|
|    scientist|   31|
|      student|  196|
|     salesman|   12|
|       artist|   28|
|   technician|   27|
|administrator|   79|
|     engineer|   67|
|   healthcare|   16|
|     educator|   95|
|entertainment|   18|
|    homemaker|    7|
+-------------+-----+
only showing top 20 rows



### Problem 4: Adding a New Column
Problem: Add a new column "age_group" to the occupation DataFrame based on the age of the users. Divide users into age groups: "18-25", "26-35", "36-50", and "51+".

In [21]:
occupation.withColumn("age_group",
                      when((occupation.age>=18) & (occupation.age<=25), lit("18-25"))\
                      .when((occupation.age>=26) & (occupation.age<=35), lit("18-25"))\
                      .when((occupation.age>=36) & (occupation.age<=50), lit("18-25"))\
                      .otherwise(lit("51+"))\
                        ).show()

+-------+---+------+-------------+--------+---------+
|user_id|age|gender|   occupation|zip_code|age_group|
+-------+---+------+-------------+--------+---------+
|      1| 24|     M|   technician|   85711|    18-25|
|      2| 53|     F|        other|   94043|      51+|
|      3| 23|     M|       writer|   32067|    18-25|
|      4| 24|     M|   technician|   43537|    18-25|
|      5| 33|     F|        other|   15213|    18-25|
|      6| 42|     M|    executive|   98101|    18-25|
|      7| 57|     M|administrator|   91344|      51+|
|      8| 36|     M|administrator|   05201|    18-25|
|      9| 29|     M|      student|   01002|    18-25|
|     10| 53|     M|       lawyer|   90703|      51+|
|     11| 39|     F|        other|   30329|    18-25|
|     12| 28|     F|        other|   06405|    18-25|
|     13| 47|     M|     educator|   29206|    18-25|
|     14| 45|     M|    scientist|   55106|    18-25|
|     15| 49|     F|     educator|   97301|    18-25|
|     16| 21|     M|entertai

### Problem 5: Creating DataFrames and Converting to Spark Types
Problem: Given the provided code snippet, create a DataFrame df using the given data and schema. The schema includes columns for firstname, middlename, lastname, id, gender, and salary. After creating the DataFrame, print its schema and display its content without truncation.

In [26]:
# Define the schema
schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

# Sample data
data = [
    ("James", "", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", "", "40288", "M", 4000),
    ("Robert", "", "Williams", "42114", "M", 4000),
    ("Maria", "Anne", "Jones", "39192", "F", 4000),
    ("Jen", "Mary", "Brown", "", "F", -1)
]

# Create a DataFrame
df = spark.createDataFrame(data, schema)

# Print the schema
df.printSchema()

# Display the content without truncation
df.show(truncate=False)



root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



### Problem 6: Adding and Renaming Columns
Problem: Add a new column "gender" to the existing DataFrame and rename the "Age" column to "Years".

In [31]:
occupation=occupation.withColumn("gender")
occupation=occupation.withColumnRenamed("age","Years")
occupation.show()

+-------+-----+------+-------------+--------+
|user_id|Years|gender|   occupation|zip_code|
+-------+-----+------+-------------+--------+
|      1|   24|     M|   technician|   85711|
|      2|   53|     F|        other|   94043|
|      3|   23|     M|       writer|   32067|
|      4|   24|     M|   technician|   43537|
|      5|   33|     F|        other|   15213|
|      6|   42|     M|    executive|   98101|
|      7|   57|     M|administrator|   91344|
|      8|   36|     M|administrator|   05201|
|      9|   29|     M|      student|   01002|
|     10|   53|     M|       lawyer|   90703|
|     11|   39|     F|        other|   30329|
|     12|   28|     F|        other|   06405|
|     13|   47|     M|     educator|   29206|
|     14|   45|     M|    scientist|   55106|
|     15|   49|     F|     educator|   97301|
|     16|   21|     M|entertainment|   10309|
|     17|   30|     M|   programmer|   06355|
|     18|   35|     F|        other|   37212|
|     19|   40|     M|    libraria

### Problem 7: Filtering Rows and Sorting
Problem: Filter out users who are younger than 30 years and sort the DataFrame by age in descending order.

In [44]:
occupation.select("*").filter(col("age")<30).orderBy(desc("age")).show()

+-------+-----+------+----------+--------+
|user_id|Years|gender|occupation|zip_code|
+-------+-----+------+----------+--------+
|    222|   29|     M|programmer|   27502|
|     43|   29|     F| librarian|   20854|
|     45|   29|     M|programmer|   50233|
|    109|   29|     M|     other|   55423|
|    193|   29|     M|   student|   49931|
|      9|   29|     M|   student|   01002|
|    229|   29|     F| librarian|   22903|
|    250|   29|     M| executive|   95110|
|    297|   29|     F|  educator|   98103|
|    299|   29|     M|    doctor|   63108|
|    354|   29|     F| librarian|   48197|
|    365|   29|     M|    lawyer|   20009|
|    404|   29|     F|programmer|   55108|
|    407|   29|     M|  engineer|   03261|
|    467|   29|     M|  engineer|   91201|
|    473|   29|     M|   student|   94708|
|    478|   29|     M|     other|   10019|
|    483|   29|     M| scientist|   43212|
|    490|   29|     F|    artist|   V5A2B|
|    495|   29|     M|  engineer|   03052|
+-------+--

### Problem 8: Repartitioning and Collecting Rows
Problem: Repartition the DataFrame into 2 partitions without shuffling the data, then collect and display all rows in the driver and print number of partitions

In [58]:
# Repartition the DataFrame into 2 partitions without shuffling
df_repartitioned = df.repartition(2)
# Collect and display all rows in the driver
rows = df_repartitioned.collect()
for row in rows:
    print(row)

Row(firstname='James', middlename='', lastname='Smith', id='36636', gender='M', salary=3000)
Row(firstname='Michael', middlename='Rose', lastname='', id='40288', gender='M', salary=4000)
Row(firstname='Robert', middlename='', lastname='Williams', id='42114', gender='M', salary=4000)
Row(firstname='Maria', middlename='Anne', lastname='Jones', id='39192', gender='F', salary=4000)
Row(firstname='Jen', middlename='Mary', lastname='Brown', id='', gender='F', salary=-1)


In [59]:
# Print the number of partitions
num_partitions = df_repartitioned.rdd.getNumPartitions()
print("Number of partitions = ", num_partitions)

Number of partitions =  2


### Additional questions:

Use both spark SQL and Pyspark to obtain answer wherever relevant

#### Filter out rows where the age is greater than 30 and create a new DataFrame. Then, add a new column named "is_elderly" with a value of "True" for these rows and "False" otherwise.Rename the "gender" column to "sex".

In [67]:
# Spark SQL
occupation.createOrReplaceTempView("people")
query="""SELECT *,\
           CASE WHEN age > 30 THEN True ELSE False END as is_elderly,\
           gender as sex\
    FROM people\
    WHERE age > 30"""
spark.sql(query).show()

+-------+---+------+-------------+--------+----------+---+
|user_id|age|gender|   occupation|zip_code|is_elderly|sex|
+-------+---+------+-------------+--------+----------+---+
|      2| 53|     F|        other|   94043|      true|  F|
|      5| 33|     F|        other|   15213|      true|  F|
|      6| 42|     M|    executive|   98101|      true|  M|
|      7| 57|     M|administrator|   91344|      true|  M|
|      8| 36|     M|administrator|   05201|      true|  M|
|     10| 53|     M|       lawyer|   90703|      true|  M|
|     11| 39|     F|        other|   30329|      true|  F|
|     13| 47|     M|     educator|   29206|      true|  M|
|     14| 45|     M|    scientist|   55106|      true|  M|
|     15| 49|     F|     educator|   97301|      true|  F|
|     18| 35|     F|        other|   37212|      true|  F|
|     19| 40|     M|    librarian|   02138|      true|  M|
|     20| 42|     F|    homemaker|   95660|      true|  F|
|     25| 39|     M|     engineer|   55107|      true|  

In [64]:
# Pyspark
# Filter out rows where age is greater than 30
filtered_occupation = occupation.filter(occupation["age"] > 30)

# Add a new column "is_elderly" based on age
result_occupation = filtered_occupation.withColumn("is_elderly", 
when(occupation.age > 30, "True").otherwise("False"))

# Rename the "gender" column to "sex"
result_occupation = result_occupation.withColumnRenamed("gender", "sex")

result_occupation.show()

+-------+---+---+-------------+--------+----------+
|user_id|age|sex|   occupation|zip_code|is_elderly|
+-------+---+---+-------------+--------+----------+
|      2| 53|  F|        other|   94043|      True|
|      5| 33|  F|        other|   15213|      True|
|      6| 42|  M|    executive|   98101|      True|
|      7| 57|  M|administrator|   91344|      True|
|      8| 36|  M|administrator|   05201|      True|
|     10| 53|  M|       lawyer|   90703|      True|
|     11| 39|  F|        other|   30329|      True|
|     13| 47|  M|     educator|   29206|      True|
|     14| 45|  M|    scientist|   55106|      True|
|     15| 49|  F|     educator|   97301|      True|
|     18| 35|  F|        other|   37212|      True|
|     19| 40|  M|    librarian|   02138|      True|
|     20| 42|  F|    homemaker|   95660|      True|
|     25| 39|  M|     engineer|   55107|      True|
|     26| 49|  M|     engineer|   21044|      True|
|     27| 40|  F|    librarian|   30030|      True|
|     28| 32

#### Calculate the average age of male and female users separately. Present the result in a new DataFrame with columns "gender" and "avg_age".

In [70]:
# Spark SQL
occupation.createOrReplaceTempView("people")
query="""select gender,avg(age) as avg_age
from people
group by gender"""
spark.sql(query).show()




+------+------------------+
|gender|           avg_age|
+------+------------------+
|     F| 33.81318681318681|
|     M|34.149253731343286|
+------+------------------+



In [75]:
# Pyspark

avg_age_df = occupation.groupBy("gender").agg(avg("age").alias("avg_age"))
avg_age_df.show()


+------+------------------+
|gender|           avg_age|
+------+------------------+
|     F| 33.81318681318681|
|     M|34.149253731343286|
+------+------------------+



#### Add a new column named "full_name" to the dataset by concatenating the "user_id" and "occupation" columns. Then, rename the "zip_code" column to "postal_code" in the same DataFrame.

In [104]:
# Spark SQL
occupation.createOrReplaceTempView("new_occupation")
update_query="""ALTER VIEW  new_occupation
RENAME COLUMN zip_code TO postal_code"""
spark.sql(update_query)



ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'COLUMN': missing 'TO'.(line 2, pos 7)

== SQL ==
ALTER VIEW  new_occupation
RENAME COLUMN zip_code TO postal_code
-------^^^


In [88]:
query="""select *, CONCAT(user_id,occupation) as full_name
from new_occupation
"""
spark.sql(query).show()

+-------+---+------+-------------+--------+---------------+
|user_id|age|gender|   occupation|zip_code|      full_name|
+-------+---+------+-------------+--------+---------------+
|      1| 24|     M|   technician|   85711|    1technician|
|      2| 53|     F|        other|   94043|         2other|
|      3| 23|     M|       writer|   32067|        3writer|
|      4| 24|     M|   technician|   43537|    4technician|
|      5| 33|     F|        other|   15213|         5other|
|      6| 42|     M|    executive|   98101|     6executive|
|      7| 57|     M|administrator|   91344| 7administrator|
|      8| 36|     M|administrator|   05201| 8administrator|
|      9| 29|     M|      student|   01002|       9student|
|     10| 53|     M|       lawyer|   90703|       10lawyer|
|     11| 39|     F|        other|   30329|        11other|
|     12| 28|     F|        other|   06405|        12other|
|     13| 47|     M|     educator|   29206|     13educator|
|     14| 45|     M|    scientist|   551

In [92]:
# Pyspark
# Add a new column "full_name" by concatenating "user_id" and "occupation"
df_with_full_name = occupation.withColumn("full_name", concat(col("user_id"), col("occupation")))

# Rename the "zip_code" column to "postal_code"
df_renamed = df_with_full_name.withColumnRenamed("zip_code", "postal_code")

df_renamed.show()

+-------+---+------+-------------+-----------+---------------+
|user_id|age|gender|   occupation|postal_code|      full_name|
+-------+---+------+-------------+-----------+---------------+
|      1| 24|     M|   technician|      85711|    1technician|
|      2| 53|     F|        other|      94043|         2other|
|      3| 23|     M|       writer|      32067|        3writer|
|      4| 24|     M|   technician|      43537|    4technician|
|      5| 33|     F|        other|      15213|         5other|
|      6| 42|     M|    executive|      98101|     6executive|
|      7| 57|     M|administrator|      91344| 7administrator|
|      8| 36|     M|administrator|      05201| 8administrator|
|      9| 29|     M|      student|      01002|       9student|
|     10| 53|     M|       lawyer|      90703|       10lawyer|
|     11| 39|     F|        other|      30329|        11other|
|     12| 28|     F|        other|      06405|        12other|
|     13| 47|     M|     educator|      29206|     13ed

#### Filter out rows where occupation is 'technician', select only the "user_id" and "age" columns, and then add a new column "age_diff" that calculates the difference between the user's age and the average age in the dataset.

In [99]:
# Spark SQL
occupation.createOrReplaceTempView("new_occupation")
query="""select user_id,age,age - (SELECT AVG(age) FROM new_occupation WHERE occupation == 'technician') AS age_diff
 from new_occupation where occupation="technician"
"""
spark.sql(query).show()


+-------+---+-------------------+
|user_id|age|           age_diff|
+-------+---+-------------------+
|      1| 24| -9.148148148148145|
|      4| 24| -9.148148148148145|
|     44| 26| -7.148148148148145|
|     77| 30|-3.1481481481481453|
|    143| 42|  8.851851851851855|
|    197| 55| 21.851851851851855|
|    244| 28| -5.148148148148145|
|    294| 34| 0.8518518518518547|
|    311| 32|-1.1481481481481453|
|    325| 48| 14.851851851851855|
|    441| 50| 16.851851851851855|
|    456| 24| -9.148148148148145|
|    458| 47| 13.851851851851855|
|    488| 48| 14.851851851851855|
|    545| 27| -6.148148148148145|
|    670| 30|-3.1481481481481453|
|    715| 21|-12.148148148148145|
|    717| 24| -9.148148148148145|
|    718| 42|  8.851851851851855|
|    738| 35| 1.8518518518518547|
+-------+---+-------------------+
only showing top 20 rows



In [101]:
# Pyspark
filtered_df = occupation.filter(col("occupation") == "technician")

selected_df = filtered_df.select("user_id", "age")

average_age = occupation.select(avg("age")).first()[0]

result_df = selected_df.withColumn("age_diff", col("age") - average_age)

result_df.show()


+-------+---+-------------------+
|user_id|age|           age_diff|
+-------+---+-------------------+
|      1| 24|-10.051961823966067|
|      4| 24|-10.051961823966067|
|     44| 26| -8.051961823966067|
|     77| 30| -4.051961823966067|
|    143| 42|  7.948038176033933|
|    197| 55| 20.948038176033933|
|    244| 28| -6.051961823966067|
|    294| 34|-0.0519618239660673|
|    311| 32|-2.0519618239660673|
|    325| 48| 13.948038176033933|
|    441| 50| 15.948038176033933|
|    456| 24|-10.051961823966067|
|    458| 47| 12.948038176033933|
|    488| 48| 13.948038176033933|
|    545| 27| -7.051961823966067|
|    670| 30| -4.051961823966067|
|    715| 21|-13.051961823966067|
|    717| 24|-10.051961823966067|
|    718| 42|  7.948038176033933|
|    738| 35| 0.9480381760339327|
+-------+---+-------------------+
only showing top 20 rows



#### Divide the dataset into two DataFrames: one with male users and another with female users. Repartition both DataFrames to have 2 partitions each. Then, union these two DataFrames together and display the resulting DataFrame.

In [102]:
male_df = df.filter(df["gender"] == "M")
female_df = df.filter(df["gender"] == "F")

repartitioned_df_male=male_df.repartition(2)
repartitioned_df_female=female_df.repartition(2)

combined_df = male_df.union(female_df)

combined_df.show()


+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+



#### Create and fill a new DataFrame named user_ratings with columns user_id and rating max 10 column. Both user_data and user_ratings share the user_id column. Combine these two DataFrames to create a new DataFrame that includes user information and their corresponding ratings. Make sure to keep only the users present in both DataFrames.

In [103]:
user_data_schema = "user_id INT, name STRING, age INT"

user_data = [
    (1, "John", 30),
    (2, "Jane", 25),
    (3, "Michael", 40),
    (4, "Emily", 35),
    (5, "David", 28)
]

user_data_df = spark.createDataFrame(user_data, schema=user_data_schema)

user_ratings_schema = "user_id INT, rating INT"

user_ratings = [
    (1, 8),
    (2, 9),
    (3, 7),
    (5, 10)
]

user_ratings_df = spark.createDataFrame(user_ratings, schema=user_ratings_schema)

combined_df = user_data_df.join(user_ratings_df, on="user_id")

combined_df.show()


+-------+-------+---+------+
|user_id|   name|age|rating|
+-------+-------+---+------+
|      1|   John| 30|     8|
|      2|   Jane| 25|     9|
|      3|Michael| 40|     7|
|      5|  David| 28|    10|
+-------+-------+---+------+

